In [40]:
import boto3
import os

# create an s3 instance object
s3 = boto3.resource('s3',
                    aws_access_key_id=os.getenv('AWS_ACCESS_KEY'),
                    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY') 
                   )

In [42]:
# create a busket
try:
    s3.create_bucket(Bucket='14848-hw3-bucket', CreateBucketConfiguration={
        'LocationConstraint': 'us-west-1'}) 
except Exception as e:
    print (e)
    
bucket = s3.Bucket("14848-hw3-bucket")
bucket.Acl().put(ACL='public-read')

An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


{'ResponseMetadata': {'RequestId': 'DCZGXXA424DXQW6R',
  'HostId': 'Xo6y8NJDTz6jk5+kAVmOX4XP2k8WOvZuJa+5dGKhaKgvDsuj89XazqP5JLLwepXdFqvYt9WG0Ws=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Xo6y8NJDTz6jk5+kAVmOX4XP2k8WOvZuJa+5dGKhaKgvDsuj89XazqP5JLLwepXdFqvYt9WG0Ws=',
   'x-amz-request-id': 'DCZGXXA424DXQW6R',
   'date': 'Wed, 20 Oct 2021 00:03:01 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 1}}

In [43]:
# upload experiments.csv into busket
body = open('datasets/experiments.csv', 'rb')
o = s3.Object('14848-hw3-bucket', 'experiments.csv').put(Body=body)
s3.Object('14848-hw3-bucket', 'experiments.csv').Acl().put(ACL='public-read')

{'ResponseMetadata': {'RequestId': '2PQSPR9AZX9GD75A',
  'HostId': 'Fo1oSuxkMd5ToEFbDlAzQIzfy+FoAxdiRSCSYcCVkjAtqUGQkY+r05cINdxTReUmTyVCytGhFKA=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Fo1oSuxkMd5ToEFbDlAzQIzfy+FoAxdiRSCSYcCVkjAtqUGQkY+r05cINdxTReUmTyVCytGhFKA=',
   'x-amz-request-id': '2PQSPR9AZX9GD75A',
   'date': 'Wed, 20 Oct 2021 00:03:02 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [44]:
# create dynamo db resource
dyndb = boto3.resource('dynamodb',
                       region_name='us-west-1',
                       aws_access_key_id=os.getenv('AWS_ACCESS_KEY'),
                       aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY') 
)

In [45]:
# create table in dynamo db
try:
    table = dyndb.create_table(
        TableName='DataTable',
        KeySchema=[
            {
                'AttributeName': 'PartitionKey',
                'KeyType': 'HASH'
            }, {
                'AttributeName': 'RowKey',
                'KeyType': 'RANGE'
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'PartitionKey',
                'AttributeType': 'S'
            }, {
                'AttributeName': 'RowKey',
                'AttributeType': 'S'
            },
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5
        }
)
except Exception as e:
    print (e)
    table = dyndb.Table("DataTable")
    
table.meta.client.get_waiter('table_exists').wait(TableName='DataTable')

An error occurred (ResourceInUseException) when calling the CreateTable operation: Table already exists: DataTable


In [46]:
import csv

# columns in experiments.csv: Id, Temp, Conductivity, Concentration, URL
with open('datasets/experiments.csv', 'rt') as csvfile: 
    csvf = csv.reader(csvfile, delimiter=',', quotechar='|')
    
    # skips the first row of the CSV file.
    next(csvf)
    
    for item in csvf:
        print (item)
        
        # upload exp1, 2, 3 data to s3 bucket
        body = open('datasets/'+item[4], 'rb') 
        s3.Object('14848-hw3-bucket', item[4]).put(Body=body )
        md = s3.Object('14848-hw3-bucket', item[4]).Acl().put(ACL='public-read')

        url = "https://s3-us-west-1.amazonaws.com/14848-hw3-bucket/" + item[4] 

        
        # put items in the table
        partition_key = item[4].split('.csv')[0]
        metadata_item = {'PartitionKey': partition_key, 'RowKey': item[0], 
                     'temp' : item[1], 'conductivity' : item[2], 'concentration' : item[3], 'url':url}
        
        try: 
            table.put_item(Item=metadata_item)
        except:
            print ("item may already be there or another failure")

['1', '-1', '52', '3.4', 'exp1.csv']
['2', '-2', '52.1', '3.4', 'exp2.csv']
['3', '-2.93', '57.1', '3.7', 'exp3.csv']


In [47]:
print("Fetch PartitionKey exp1")
response = table.get_item(
    Key={
        'PartitionKey': 'exp1',
        'RowKey': '1'
    }
)
item = response['Item'] 

print(item)
print()
print(response)
print()

Fetch PartitionKey exp1
{'temp': '-1', 'RowKey': '1', 'conductivity': '52', 'PartitionKey': 'exp1', 'concentration': '3.4', 'url': 'https://s3-us-west-1.amazonaws.com/14848-hw3-bucket/exp1.csv'}

{'Item': {'temp': '-1', 'RowKey': '1', 'conductivity': '52', 'PartitionKey': 'exp1', 'concentration': '3.4', 'url': 'https://s3-us-west-1.amazonaws.com/14848-hw3-bucket/exp1.csv'}, 'ResponseMetadata': {'RequestId': 'OGHPI6BJOJBLGDRRICH1057413VV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Wed, 20 Oct 2021 00:03:41 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '204', 'connection': 'keep-alive', 'x-amzn-requestid': 'OGHPI6BJOJBLGDRRICH1057413VV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '2891708705'}, 'RetryAttempts': 0}}



In [48]:
print("Fetch PartitionKey exp2 \n")
response = table.get_item(
    Key={
        'PartitionKey': 'exp2',
        'RowKey': '2'
    }
)
item = response['Item'] 

print(item)
print()
print(response)
print()

Fetch PartitionKey exp2 

{'temp': '-2', 'RowKey': '2', 'conductivity': '52.1', 'PartitionKey': 'exp2', 'concentration': '3.4', 'url': 'https://s3-us-west-1.amazonaws.com/14848-hw3-bucket/exp2.csv'}

{'Item': {'temp': '-2', 'RowKey': '2', 'conductivity': '52.1', 'PartitionKey': 'exp2', 'concentration': '3.4', 'url': 'https://s3-us-west-1.amazonaws.com/14848-hw3-bucket/exp2.csv'}, 'ResponseMetadata': {'RequestId': 'SLTPELPU5IJRD1DSNJ0LTSJIR7VV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Wed, 20 Oct 2021 00:03:42 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '206', 'connection': 'keep-alive', 'x-amzn-requestid': 'SLTPELPU5IJRD1DSNJ0LTSJIR7VV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '1371533744'}, 'RetryAttempts': 0}}



In [49]:
print("Fetch PartitionKey exp3 \n")
response = table.get_item(
    Key={
        'PartitionKey': 'exp3',
        'RowKey': '3'
    }
)
item = response['Item'] 

print(item)
print()
print(response)
print()

Fetch PartitionKey exp3 

{'temp': '-2.93', 'RowKey': '3', 'conductivity': '57.1', 'PartitionKey': 'exp3', 'concentration': '3.7', 'url': 'https://s3-us-west-1.amazonaws.com/14848-hw3-bucket/exp3.csv'}

{'Item': {'temp': '-2.93', 'RowKey': '3', 'conductivity': '57.1', 'PartitionKey': 'exp3', 'concentration': '3.7', 'url': 'https://s3-us-west-1.amazonaws.com/14848-hw3-bucket/exp3.csv'}, 'ResponseMetadata': {'RequestId': 'D5AJFF12RT1EP8RLT96RS2AKANVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Wed, 20 Oct 2021 00:03:42 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '209', 'connection': 'keep-alive', 'x-amzn-requestid': 'D5AJFF12RT1EP8RLT96RS2AKANVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '1819576774'}, 'RetryAttempts': 0}}

